## 数据处理

In [1]:
import pandas as pd
import io
import re
from datetime import datetime, timedelta
from typing import List, Union, Optional
from datetime import datetime

import os, re, json, time
import typing as T
import requests
from pathlib import Path 
from openpyxl import Workbook, load_workbook

from openpyxl.utils import get_column_letter
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from data_processing import load_and_process,build_jsonl_for_range, save_jsonl
from model_classifyV1_Copy1 import postprocess_excel_by_topic

In [2]:
## 研发字典
speaker_map = {
    "16186514":   "peter本尊",
    "1655611808": "运营绾绾",
    "2073820674": "沙利文老师",
    "2726067525": "milissa",
}
## 客服字典
MAPPING_FILE = "mapping地球1.xlsx"

##QQ的txt文件
pathtxt   = "1205《欢迎来到地球》测试1群.txt"

# 设定时间范围
start_time = "2025-12-03 14:00:00"
end_time   = "2025-12-03 15:00:00"


# 1) 拿到 JSONL（列表）
jsonl_lines01 = build_jsonl_for_range(
    pathtxt=pathtxt,
    mapping_file=MAPPING_FILE,
    speaker_map=speaker_map,
    start_time=start_time,
    end_time=end_time,
    return_str=False,   # 返回 list[str]
)

print(len(jsonl_lines01), "lines")
print(jsonl_lines01[0])



290 lines
{"发言日期": "2025-12-03", "发言时间": "14:00:00", "玩家ID": "何方路人甲(978617207)", "玩家消息": "我第一次玩就是跟怪诞虫和残肢"}


## 大模型分类

## 定义

In [3]:
"""
批处理 10000 条聊天数据（每批 100 条）：
- 模型#1：过滤非游戏相关（只保留相关 JSON 行，原样输出）
- 模型#2：对保留内容进行 5 类意图分类（输出带“意图分类”的 JSON 行）
- 结果按 sheet（体感反馈/疑惑不解和问题询问/玩家建议和灵感/情绪输出/问题反馈）写入 Excel
"""
from model_classifyV1_Copy1 import (
    load_system_prompt,
    build_user_prompt_filter, build_user_prompt_classify,build_user_prompt_classify2,
    call_ark_chat_completions,
    jsonl_to_dataframe_with_intent,
    create_intent_excel_styled, append_json_to_excel_by_cat_and_tag,load_whitelist,
    extract_clusters_from_output, update_and_save_whitelist, build_user_prompt_cluster_correct
)

## 设置参数

In [4]:
# ============= 你的模型与文件配置（改这里） =============
API_URL   = "https://ark.cn-beijing.volces.com/api/v3/chat/completions" 
API_KEY = "de91deb0-aae6-46cb-bac0-17ac3b6107f5" #API
V3_MODEL_ID= "ep-20251020160142-5d7hp"#接入点
V3_1_MODEL_ID = "ep-20251020160025-9p5tj"#接入点
R1_MODEL_ID = "ep-20251020160103-5n6g2"#接入点
                 
PROMPT_MD_PATH01 = Path("提示词1.md")      # 模型#1 system 提示词（筛相关）
PROMPT_MD_PATH02 = Path("提示词2.md")      # 模型#2 system 提示词（做分类）
PROMPT_MD_PATH03 = Path ("提示词5.md")     # 模型#3 system 提示词（做话提簇）
PROMPT_MD_PATH04 = Path ("话提簇校正提示词.md")     # 模型#3 system 提示词（做话提簇修正）

EXCEL_FILE       = Path("12041群.xlsx")   # 输出 Excel
BATCH_SIZE       = 200
SLEEP_BETWEEN    = 1   # 每批之间的间隔，防止QPS触发限流；按需调整
RETRIES          = 2
TEMPERATURE      = 0.20
MAX_TOKENS       = 16384
TIMEOUT_SEC      = 600
# =====================================================

## 开始运行

In [5]:
from tqdm import tqdm
import time
import pandas as pd
import json

# ... 前置：系统提示、jsonl_lines01 等

system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 做分类
system_prompt03 = load_system_prompt(PROMPT_MD_PATH03)  # 做二级标签
# ❌ 不再需要模型4
# system_prompt04 = load_system_prompt(PROMPT_MD_PATH04)

# ❌ 不需要建 Excel、不需要白名单
# create_intent_excel_styled(EXCEL_FILE)
# WHITE_LIST_PATH = Path("话题簇白名单.jsonl")
# ...

total = len(jsonl_lines01)
if total == 0:
    print("没有可处理的数据。")
else:
    BATCH_SIZE = BATCH_SIZE  # 确保你已有这个常量
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")

written_total = 0

# ⭐ 收集所有批次的 df（模型3的结果）
all_dfs = []

for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch_lines = jsonl_lines01[start:end]

    try:
        # --- 模型 #1：筛相关 ---
        user_prompt1 = build_user_prompt_filter(batch_lines)
        output_filter = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt01,
            user_prompt=user_prompt1,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        if not output_filter:
            tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
            continue

        # --- 模型 #2：五分类 ---
        user_prompt2 = build_user_prompt_classify(output_filter)
        output_classify = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt02,
            user_prompt=user_prompt2,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        if not output_classify:
            tqdm.write(f"[批次 {b+1}] 模型#2 无有效输出，跳过。")
            continue

        # --- 模型 #3：二级标签 & 话题簇（初版）---
        user_prompt3 = build_user_prompt_classify2(output_classify)
        output_classify2 = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt03,
            user_prompt=user_prompt3,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        print(output_classify2)
        # ❌ 不再做模型4：话题簇校正
        # user_prompt4 = build_user_prompt_cluster_correct(output_classify2, whitelist)
        # output_classify3 = ...

        # --- 直接把模型3的结果转成 DataFrame ---
        df_batch = jsonl_to_dataframe_with_intent(output_classify2)
        if df_batch is None or df_batch.empty:
            tqdm.write(f"[批次 {b+1}] ⚠️ 无可用数据（空 DataFrame）。")
            continue

        all_dfs.append(df_batch)
        batch_written = len(df_batch)
        written_total += batch_written
        tqdm.write(f"[批次 {b+1}] ✅ 收集 {batch_written} 条（模型3结果）。")

    except Exception as e:
        tqdm.write(f"[批次 {b+1}] ❌ 出错：{e}")
        continue

    time.sleep(SLEEP_BETWEEN)

# ❌ 不再做 Excel 后处理
# postprocess_excel_by_topic(...)

print("\n✅ 全部批次处理完成！")
print(f"输入总数：{total}")
print(f"有效结果总数（模型3行数）：{written_total}")



准备处理 290 条，共 2 批（每批 200 条）。


🔥 批处理进度:   0%|                                                                                                                                     | 0/2 [10:35<?, ?批/s]      

{"发言日期": "2025-12-03", "发言时间": "14:00:00", "玩家ID": "何方路人甲(978617207)", "玩家消息": "我第一次玩就是跟怪诞虫和残肢", "分类标签": 1, "话题簇": "开局道具", "描述": "初始装备与道具选择讨论"}
{"发言日期": "2025-12-03", "发言时间": "14:00:09", "玩家ID": "何方路人甲(978617207)", "玩家消息": "那时候还不会摆道具", "分类标签": 1, "话题簇": "开局道具", "描述": "初始装备与道具选择讨论"}
{"发言日期": "2025-12-03", "发言时间": "14:00:15", "玩家ID": "何方路人甲(978617207)", "玩家消息": "一直疯", "分类标签": 4, "话题簇": "泛情绪簇", "描述": "无法归类的情绪表达"}
{"发言日期": "2025-12-03", "发言时间": "14:00:17", "玩家ID": "枫火(276465118)", "玩家消息": "又是玩球", "分类标签": 1, "话题簇": "开局道具", "描述": "初始装备与道具选择讨论"}
{"发言日期": "2025-12-03", "发言时间": "14:00:30", "玩家ID": "甲乙丙丁(449511)", "玩家消息": "又是西洋剑", "分类标签": 1, "话题簇": "开局道具", "描述": "初始装备与道具选择讨论"}
{"发言日期": "2025-12-03", "发言时间": "14:00:48", "玩家ID": "空(3316920376)", "玩家消息": "啊要重新下载", "分类标签": 4, "话题簇": "客户端问题", "描述": "游戏安装与更新问题"}
{"发言日期": "2025-12-03", "发言时间": "14:00:53", "玩家ID": "尘(3411480843)", "玩家消息": "开PVP了", "分类标签": 1, "话题簇": "玩法模式", "描述": "游戏模式与玩法类型讨论"}
{"发言日期": "2025-12-03", "发言时间": "14:00:59", "玩家ID": "甲乙丙丁(449

🔥 批处理进度:  50%|██████████████████████████████████████████████████████████████                                                              | 1/2 [17:34<10:36, 636.55s/批]      

{"发言日期": "2025-12-03", "发言时间": "14:30:38", "玩家ID": "天亮说晚安(786351340)", "玩家消息": "佬们，现在开局选哪个殖装好?", "分类标签": 2, "话题簇": "开局选择", "描述": "游戏初始角色或装备的选择建议"}
{"发言日期": "2025-12-03", "发言时间": "14:30:40", "玩家ID": "ۣۖิ戏ۣۖิ子(1046996232)", "玩家消息": "@青瓷客服-大炮 殖装卡没了", "分类标签": 5, "话题簇": "殖装丢失", "描述": "游戏内殖装装备异常消失问题"}
{"发言日期": "2025-12-03", "发言时间": "14:30:49", "玩家ID": "寒蝉·彼岸(416459252)", "玩家消息": "现在不用练级了", "分类标签": 1, "话题簇": "等级机制", "描述": "角色等级系统与升级机制变化"}
{"发言日期": "2025-12-03", "发言时间": "14:30:53", "玩家ID": "寒蝉·彼岸(416459252)", "玩家消息": "起手满级", "分类标签": 1, "话题簇": "等级机制", "描述": "角色等级系统与升级机制变化"}
{"发言日期": "2025-12-03", "发言时间": "14:31:58", "玩家ID": "ۣۖิ戏ۣۖิ子(1046996232)", "玩家消息": "对啊但是殖装应该都在吧", "分类标签": 2, "话题簇": "殖装丢失", "描述": "游戏内殖装装备异常消失问题"}
{"发言日期": "2025-12-03", "发言时间": "14:32:13", "玩家ID": "枫火(276465118)", "玩家消息": "驾驶员上限被削了一个", "分类标签": 1, "话题簇": "角色限制", "描述": "驾驶员数量或使用限制调整"}
{"发言日期": "2025-12-03", "发言时间": "14:32:14", "玩家ID": "uneko(2724118501)", "玩家消息": "[图片]", "分类标签": 5, "话题簇": "殖装丢失", "描述": "游戏内殖装装备异常消失问题"}
{"发言日期

🔥 批处理进度: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [17:35<00:00, 527.65s/批]


✅ 全部批次处理完成！
输入总数：290
有效结果总数（模型3行数）：179
